# Flights

Om het bestand met flights bruikbaar te maken voor ArangoDB, moeten er eerst een aantal zaken aangepast worden. Zo moet de kolom id bijvoorbeeld hernoemd worden naar _key.

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:/Users/bmpka/Documents/Thomas More/3 BIT/Big Data/Schiphol/downloads/flight.csv")
df.head() #Show the top 5 rows

,id,schipholId,flightName,scheduleDate,flightDirection,flightNumber,prefixIATA,prefixICAO,scheduleTime,serviceType,...,actualOffBlockTime,terminal,gate,expectedTimeOnBelt,aircraftRegistration,airlineCode,expectedTimeGateOpen,expectedTimeBoarding,expectedTimeGateClosing,schemaVersion
0,1,122761158488203773,KL3104,2017-10-01,D,3104.0,KL,KLM,00:05:00,J,...,2017-10-01 00:05:03,3.0,G06,NaN,TCNBD,100.0,NaN,NaN,NaN,3
1,2,122761158109842734,HV5764,2017-10-01,A,5764.0,HV,TRA,00:05:00,J,...,NaN,1.0,C15,2017-10-01 00:33:04,PHHXI,164.0,NaN,NaN,NaN,3
2,3,122761157517802478,DL7494,2017-10-01,A,7494.0,DL,DAL,00:05:00,J,...,NaN,1.0,C15,2017-10-01 00:33:04,PHHXI,58.0,NaN,NaN,NaN,3
3,4,122761158488113936,KL2574,2017-10-01,A,2574.0,KL,KLM,00:05:00,J,...,NaN,1.0,C15,2017-10-01 00:33:04,PHHXI,100.0,NaN,NaN,NaN,3
4,5,122761159147708945,PC1256,2017-10-01,D,1256.0,PC,PGT,00:05:00,J,...,2017-10-01 00:05:03,3.0,G06,NaN,TCNBD,139.0,NaN,NaN,NaN,3


In [7]:
# kolom id hernoemen naar _key
df=df.rename(columns = {'id':'_key'})

# flightNumber en airlineCode converteren naar int
df['flightNumber'] = df['flightNumber'].fillna(0).astype(int)
df['airlineCode'] = df['airlineCode'].fillna(0).astype(int)

Er worden nog enkele kolommen toegevoegd aan de hand van bestaande data om mee te kunnen werken later.

In [8]:
df['scheduleTimestamp'] = df['scheduleDate'].astype(str) + ' ' + df['scheduleTime'].astype(str)
#if df['flightDirection'].astype(str) == 'A':
#if df['actualLandingTime'] != 'null':
df['delayLanding'] = pd.to_datetime(df['actualLandingTime']) - pd.to_datetime(df['scheduleTimestamp'])

df['delayDeparture'] = pd.to_datetime(df['actualOffBlockTime']) - pd.to_datetime(df['scheduleTimestamp'])

In [11]:
# eerste twee letters van de kolom aircraftRegistration gebruiken om de landcode terug te krijgen
# voorbeelden van landcodes: PH = NL; OO = BE; TC = TR
df['aircraftCountryCode'] = df['aircraftRegistration'].str.slice(0,2)

# kolom airlineCode gebruiken voor nieuwe kolom _to
df['_to'] = df['airlineCode']
df['_from'] = df['airlineCode']

df.head() #Show the top 5 rows

,_key,schipholId,flightName,scheduleDate,flightDirection,flightNumber,prefixIATA,prefixICAO,scheduleTime,serviceType,...,expectedTimeGateOpen,expectedTimeBoarding,expectedTimeGateClosing,schemaVersion,scheduleTimestamp,delayLanding,delayDeparture,aircraftCountryCode,_to,_from
0,1,122761158488203773,KL3104,2017-10-01,D,3104,KL,KLM,00:05:00,J,...,NaN,NaN,NaN,3,2017-10-01 00:05:00,NaT,00:00:03,TC,100,100
1,2,122761158109842734,HV5764,2017-10-01,A,5764,HV,TRA,00:05:00,J,...,NaN,NaN,NaN,3,2017-10-01 00:05:00,-1 days +23:58:43,NaT,PH,164,164
2,3,122761157517802478,DL7494,2017-10-01,A,7494,DL,DAL,00:05:00,J,...,NaN,NaN,NaN,3,2017-10-01 00:05:00,-1 days +23:58:43,NaT,PH,58,58
3,4,122761158488113936,KL2574,2017-10-01,A,2574,KL,KLM,00:05:00,J,...,NaN,NaN,NaN,3,2017-10-01 00:05:00,-1 days +23:58:43,NaT,PH,100,100
4,5,122761159147708945,PC1256,2017-10-01,D,1256,PC,PGT,00:05:00,J,...,NaN,NaN,NaN,3,2017-10-01 00:05:00,NaT,00:00:03,TC,139,139


In [12]:
df.to_csv('C:/Users/bmpka/Documents/Thomas More/3 BIT/Big Data/Schiphol/downloads/flight-v2.csv')